# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-86c32028f0-## 
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Tv series explorative search") 

## Workflow 4


Consider the following exploratory scenario:


> we are interested in the TV series "How I met your mother" and we want to investigate the main aspects related to the actors and directors involved in the production, know the numerber of seasons and check what are the episodes which got the higher success/impact.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P4969`    | derivative work      | predicate |
| `wd:Q147235` | How I met your mother        | node |
| `wd:Q23831` | The Office (US)        | node |



Also consider

```
wd:Q23831 ?p ?obj .
```

is the BGP to retrieve all **properties of The Office (US)**

The workload should


1. Return the number of seasons and episodes per season of the tv series

2. Get the number of episodes in which the cast members played a role. Who are the most present actors?

3. Check who is the actor who acted in more films while working on "How I met your mother" and who is the actor who participated in more films after the end of the tv series.

4. Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members.

5. Return how many of the actors who are members of the cast of the tv series have [Kavin Bacon number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon#:~:text=Kevin%20Bacon%20himself%20has%20a,Bacon%20number%20is%20N%2B1.) equal to 2

In [2]:
# start your workflow here

In [13]:
# find predicate of the cast member ->P161
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

wd:Q23831 ?p ?o .  FILTER(!isLiteral(?o))

?p <http://schema.org/name> ?pName .

} 
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]


5

In [4]:
# find television series-> Q5398426
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

wd:Q23831 wdt:P31 ?p .  FILTER(!isLiteral(?o))

?p <http://schema.org/name> ?pName .

} 
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q5398426'), ('pName', 'television series')]


1

In [5]:
# find all tv series
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

?p wdt:P31 wd:Q5398426.

?p <http://schema.org/name> ?pName .

} 
LIMIT 5

"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1154217'), ('pName', 'Ironfist Chinmi')]
[('p', 'http://www.wikidata.org/entity/Q691528'), ('pName', 'Wallace and Gromit')]
[('p', 'http://www.wikidata.org/entity/Q8193809'), ('pName', 'Shomuni')]
[('p', 'http://www.wikidata.org/entity/Q33323'), ('pName', 'Everybody Hates Chris')]
[('p', 'http://www.wikidata.org/entity/Q500769'), ('pName', 'Freaks and Geeks')]


5

In [72]:
# find properties of tv series   (number of episodes->P1113 , number of seasons->P2437) 
# (start time -> P580 , end time -> P582)
queryString = """
SELECT DISTINCT ?p ?pName
WHERE { 

?tvseries wdt:P31 wd:Q5398426;
            ?p ?o.

?p <http://schema.org/name> ?pName .
filter regex(?pName,"start|end",'i')
} 
Order by ?p
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1110'), ('pName', 'attendance')]
[('p', 'http://www.wikidata.org/prop/direct/P2031'), ('pName', 'work period (start)')]
[('p', 'http://www.wikidata.org/prop/direct/P2032'), ('pName', 'work period (end)')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex or gender')]
[('p', 'http://www.wikidata.org/prop/direct/P2360'), ('pName', 'intended public')]
[('p', 'http://www.wikidata.org/prop/direct/P4111'), ('pName', 'danskefilm TV Christmas calendar')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pName', 'season starts')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('pName', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('pName', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P6118'), ('pName', 'season ends')]


10

In [9]:
1. Return the number of seasons and episodes per season of the tv series

In [10]:
# get number of seasons and episodes per season
queryString = """
SELECT DISTINCT ?tvseriesName ?episodes ?season
WHERE { 

    ?tvseries wdt:P31 wd:Q5398426;
              wdt:P1113 ?episodes;
              wdt:P2437 ?season.

 ?tvseries <http://schema.org/name> ?tvseriesName .

} 
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('tvseriesName', 'Everybody Hates Chris'), ('episodes', '88'), ('season', '4')]
[('tvseriesName', 'Freaks and Geeks'), ('episodes', '18'), ('season', '1')]
[('tvseriesName', 'Recess'), ('episodes', '129'), ('season', '3')]
[('tvseriesName', 'Lindenstraße'), ('episodes', '1758'), ('season', '1')]
[('tvseriesName', 'Prime Minister and I'), ('episodes', '17'), ('season', '1')]


5

In [39]:
2. Get the number of episodes in which the cast members played a role. Who are the most present actors?

In [3]:
# find chracter characters-P674
queryString = """

SELECT DISTINCT ?p ?pName
 WHERE { 

            
                       
         wd:Q147235 ?p ?o    .  
         ?p <http://schema.org/name> ?pName .
         filter regex(?pName,"character",'i')
        
        } 
  order by ?pName
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('pName', 'characters')]
[('p', 'http://www.wikidata.org/prop/direct/P1881'), ('pName', 'list of characters')]


2

In [39]:
# find the preformer ID -> P175
queryString = """

SELECT DISTINCT ?p ?pName
 WHERE { 

        
         
         wd:Q147235 wdt:P674 ?role .
                     
         ?role ?p ?o.            
         ?p <http://schema.org/name> ?pName .
         
        
        } 
  order by ?pName
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3630'), ('pName', 'Babelio author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('pName', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('pName', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('pName', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6262'), ('pName', 'Fandom article ID')]


5

In [182]:
# get the number of episodes which the cast members played a role
queryString = """

 SELECT DISTINCT ?castmemberName (sum(?episodes) as ?numberoffilms)
 
         WHERE { 
                 {
                     SELECT DISTINCT ?castmemberName ?tvseries ?episodes

                             WHERE { 
                                            ?role wdt:P175 ?castmember.
                                            ?tvseries wdt:P161 ?castmember;
                                                    wdt:P1113 ?episodes.FILTER(isNumeric(?episodes))

                                             ?castmember <http://schema.org/name> ?castmemberName .
                                             
                                             
                                           

                                    } 
            }
        }


order by desc (?castmemberName)
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('castmemberName', 'Željko Ivanek'), ('numberoffilms', '1907')]
[('castmemberName', 'Šimon Caban'), ('numberoffilms', '118')]
[('castmemberName', 'Şevval Sam'), ('numberoffilms', '110')]
[('castmemberName', 'Łukasz Simlat'), ('numberoffilms', '382')]
[('castmemberName', 'Łukasz Lewandowski'), ('numberoffilms', '3300')]


5

In [180]:
# the most present actors in films
queryString = """

 SELECT DISTINCT ?castmemberName (count(?castmemberName) as ?numberoffilms)
 
         WHERE { 
                 {
                     SELECT DISTINCT ?castmemberName ?tvseries

                             WHERE { 
                                            ?role wdt:P175 ?castmember.
                                            ?tvseries wdt:P161 ?castmember;
                                                    wdt:P1113 ?episodes.

                                             ?castmember <http://schema.org/name> ?castmemberName .

                                    } 
            }
        }

#group by ?tvseries
order by desc (?numberoffilms)
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('castmemberName', 'Claus Bue'), ('numberoffilms', '27')]
[('castmemberName', 'Kurt Ravn'), ('numberoffilms', '25')]
[('castmemberName', 'Solbjørg Højfeldt'), ('numberoffilms', '24')]
[('castmemberName', 'Henning Jensen'), ('numberoffilms', '23')]
[('castmemberName', 'Kjeld Norgaard'), ('numberoffilms', '23')]


5

In [71]:
3. Check who is the actor who acted in more films while working on "How I met your mother" and who is the actor who participated in more films after the end of the tv series.

SyntaxError: invalid syntax (3165244330.py, line 1)

In [77]:
# find start an end of "How I met your mother" (start:2005 , end: 2014)
queryString = """

SELECT DISTINCT ?start ?end
 WHERE { 

            wd:Q147235 wdt:P31 wd:Q5398426;
                       wdt:P161 ?castmember;
                       wdt:P580 ?start ;
                       wdt:P582 ?end .
                       
        } 
  
LIMIT 50

"""

print("Results")
run_query(queryString)

Results
[('start', '2005-09-19T00:00:00Z'), ('end', '2014-03-31T00:00:00Z')]


1

In [3]:
# actor who acted in more films while working on "How I met your mother"
queryString = """
SELECT DISTINCT ?castmember ?castmemberName ?start ?end
   WHERE { 
   

                    ?tvseries wdt:P31 wd:Q5398426;
                                wdt:P161 ?castmember;
                                wdt:P580 ?start ;
                                wdt:P582 ?end .

        {
          SELECT DISTINCT ?castmember 
                 WHERE { 
                        wd:Q147235 wdt:P31 wd:Q5398426;
                                     wdt:P161 ?castmember.
                        }
                        group by ?castmember

         }  

       FILTER(?start > "2005-09-19"^^xsd:dateTime && ?end < "2014-03-31"^^xsd:dateTime).
       
       ?castmember <http://schema.org/name> ?castmemberName .
                
  }              
 group by ?castmemberName

LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('castmember', 'http://www.wikidata.org/entity/Q594265'), ('castmemberName', 'Neil Jackson'), ('start', '2006-06-28T00:00:00Z'), ('end', '2006-09-13T00:00:00Z')]
[('castmember', 'http://www.wikidata.org/entity/Q454698'), ('castmemberName', 'Darcy Rose Byrnes'), ('start', '2006-09-24T00:00:00Z'), ('end', '2011-05-08T00:00:00Z')]
[('castmember', 'http://www.wikidata.org/entity/Q3943077'), ('castmemberName', 'Ryan Michelle Bathe'), ('start', '2007-09-26T00:00:00Z'), ('end', '2013-01-22T00:00:00Z')]
[('castmember', 'http://www.wikidata.org/entity/Q454698'), ('castmemberName', 'Darcy Rose Byrnes'), ('start', '2005-09-20T00:00:00Z'), ('end', '2009-05-14T00:00:00Z')]
[('castmember', 'http://www.wikidata.org/entity/Q635974'), ('castmemberName', 'Michael Trucco'), ('start', '2011-01-20T00:00:00Z'), ('end', '2012-06-15T00:00:00Z')]


5

In [4]:
# get the number of actor who acted in more films while working on "How I met your mother"
queryString = """
SELECT DISTINCT ?castmemberName  (Count(?castmember ) AS ?numberoffilm)
   WHERE { 
   

       ?tvseries wdt:P31 wd:Q5398426;
                 wdt:P161 ?castmember;
                 wdt:P580 ?start ;
                 wdt:P582 ?end .

        {
            SELECT DISTINCT ?castmember 
                 WHERE { 
                        wd:Q147235 wdt:P31 wd:Q5398426;
                                     wdt:P161 ?castmember.

                        }
                        group by ?castmember
         }  
         
         
         
       FILTER(?start > "2005-09-19"^^xsd:dateTime && ?end < "2014-03-31"^^xsd:dateTime).

       ?castmember <http://schema.org/name> ?castmemberName .
                
  }              
order by desc (?numberoffilm)
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('castmemberName', 'Amy Acker'), ('numberoffilm', '6')]
[('castmemberName', 'Virginia Williams'), ('numberoffilm', '4')]
[('castmemberName', 'Todd Stashwick'), ('numberoffilm', '4')]
[('castmemberName', 'Mircea Monroe'), ('numberoffilm', '4')]
[('castmemberName', 'Phill Lewis'), ('numberoffilm', '4')]


5

In [5]:
# get the number of actor who acted in more films after "How I met your mother"
queryString = """
SELECT DISTINCT ?castmemberName  (Count(?castmember ) AS ?numberoffilm)
   WHERE { 
   

                    ?tvseries wdt:P31 wd:Q5398426;
                                wdt:P161 ?castmember;
                                wdt:P580 ?start ;
                                wdt:P582 ?end .

        {
            SELECT DISTINCT ?castmember 
                 WHERE { 
                        wd:Q147235 wdt:P31 wd:Q5398426;
                                     wdt:P161 ?castmember.

                        }
                        group by ?castmember

         }
         

       FILTER(?start > "2014-03-31"^^xsd:dateTime).

       ?castmember <http://schema.org/name> ?castmemberName .
                
  }              
order by desc (?numberoffilm)
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('castmemberName', 'Matt Frewer'), ('numberoffilm', '4')]
[('castmemberName', 'Kal Penn'), ('numberoffilm', '4')]
[('castmemberName', 'Camryn Manheim'), ('numberoffilm', '3')]
[('castmemberName', 'Scoot McNairy'), ('numberoffilm', '3')]
[('castmemberName', 'Michael Gladis'), ('numberoffilm', '3')]


5

In [ ]:
4. Compare HIMYM with the tv series "The Office (US)" in terms of number of seasons, episods and cast members.

In [81]:
queryString = """
SELECT DISTINCT ?tvseriesName ?episodes  ?season (Count(?castmember ) AS ?numberofcastmember)
   WHERE { 
   
   Values ?tvseries{ wd:Q147235 wd:Q23831 }
   
       ?tvseries wdt:P31 wd:Q5398426;
                 wdt:P161 ?castmember;
                 wdt:P1113 ?episodes;
                 wdt:P2437 ?season.



       ?tvseries <http://schema.org/name> ?tvseriesName .
                
  }              
order by desc (?numberofcastmember)

"""

print("Results")
run_query(queryString)

Results
[('tvseriesName', 'How I Met Your Mother'), ('episodes', '208'), ('season', '9'), ('numberofcastmember', '480')]
[('tvseriesName', 'The Office'), ('episodes', '201'), ('season', '9'), ('numberofcastmember', '25')]


2

In [ ]:
5. Return how many of the actors who are members of the cast of the tv series have Kavin Bacon number equal to 2

In [186]:
# find Kevin Bacon->Q3454165
queryString = """

        SELECT DISTINCT ?castmember ?castmemberName
        WHERE { 

            ?tvseries wdt:P31 wd:Q5398426;
                       wdt:P161 ?castmember.
      
         ?castmember <http://schema.org/name> ?castmemberName .
        filter regex(?castmemberName,"Kevin Bacon",'i')

        } 

"""

print("Results")
run_query(queryString)

Results
[('castmember', 'http://www.wikidata.org/entity/Q3454165'), ('castmemberName', 'Kevin Bacon')]


1

In [247]:
#find all  members of the cast of the tv series have Kavin Bacon number equal to 1
queryString = """

        SELECT DISTINCT ?tvseriesName  ?castmemberName  
        WHERE { 

          ?tvseries wdt:P31 wd:Q5398426;
                      wdt:P161 wd:Q3454165.
                      
         ?tvseries wdt:P161 ?castmember.
         
         ?tvseries <http://schema.org/name> ?tvseriesName .
         ?castmember <http://schema.org/name> ?castmemberName .
         
         
       
        } 
order by ?castmemberName
limit 50

"""

print("Results")
run_query(queryString)

Results
[('tvseriesName', 'The Following'), ('castmemberName', 'Adán Canto')]
[('tvseriesName', 'City on a Hill'), ('castmemberName', 'Aldis Hodge')]
[('tvseriesName', 'City on a Hill'), ('castmemberName', 'Amanda Clayton')]
[('tvseriesName', 'The Following'), ('castmemberName', 'Annie Parisse')]
[('tvseriesName', 'The Following'), ('castmemberName', 'Connie Nielsen')]
[('tvseriesName', 'The Following'), ('castmemberName', 'Gregg Henry')]
[('tvseriesName', 'The Following'), ('castmemberName', 'James Purefoy')]
[('tvseriesName', 'City on a Hill'), ('castmemberName', 'Jere Shea')]
[('tvseriesName', 'City on a Hill'), ('castmemberName', 'Jeremy Rudd')]
[('tvseriesName', 'The Following'), ('castmemberName', 'Jessica Stroup')]
[('tvseriesName', 'City on a Hill'), ('castmemberName', 'Jill Hennessy')]
[('tvseriesName', 'City on a Hill'), ('castmemberName', 'Jonathan Tucker')]
[('tvseriesName', 'The Following'), ('castmemberName', 'Kevin Bacon')]
[('tvseriesName', 'City on a Hill'), ('castmemb

25

In [251]:
# find all tv series played by cast memebrs which have Kavin Bacon number equal to 1
queryString = """
SELECT DISTINCT ?tvseries ?tvseriesName
  WHERE {
                 ?tvseries wdt:P161 ?castmember.
                     {
                             SELECT DISTINCT  ?castmember 
                                      WHERE { 
                                               ?tvseries wdt:P31 wd:Q5398426;
                                                             wdt:P161 wd:Q3454165.

                                                 ?tvseries wdt:P161 ?castmember.
                                            }
                    }
            
                 
         
         ?tvseries <http://schema.org/name> ?tvseriesName .
         ?castmember <http://schema.org/name> ?castmemberName .
         
         
       
        } 
order by ?tvseriesName
limit 50

"""

print("Results")
run_query(queryString)

Results
[('tvseries', 'http://www.wikidata.org/entity/Q51236439'), ('tvseriesName', '.exe')]
[('tvseries', 'http://www.wikidata.org/entity/Q1308592'), ('tvseriesName', '100 Girls')]
[('tvseries', 'http://www.wikidata.org/entity/Q52262982'), ('tvseriesName', '11:00 pm\xa0- 12:00 am')]
[('tvseries', 'http://www.wikidata.org/entity/Q222867'), ('tvseriesName', '2 Fast 2 Furious')]
[('tvseries', 'http://www.wikidata.org/entity/Q98131145'), ('tvseriesName', '2 Hearts')]
[('tvseries', 'http://www.wikidata.org/entity/Q214013'), ('tvseriesName', '21 Grams')]
[('tvseries', 'http://www.wikidata.org/entity/Q3525931'), ('tvseriesName', '3 Days to Kill')]
[('tvseries', 'http://www.wikidata.org/entity/Q229636'), ('tvseriesName', '3 Needles')]
[('tvseries', 'http://www.wikidata.org/entity/Q276071'), ('tvseriesName', '90210')]
[('tvseries', 'http://www.wikidata.org/entity/Q108946'), ('tvseriesName', 'A Few Good Men')]
[('tvseries', 'http://www.wikidata.org/entity/Q72276'), ('tvseriesName', 'A Good Day 

50

In [253]:
#find all  members of the cast of the tv series have Kavin Bacon number equal to 2
queryString = """

SELECT DISTINCT ?castmemberName ?tvseriesName
       WHERE { 
              ?tvseries wdt:P161 ?castmember.
              {
                             SELECT DISTINCT ?tvseries
                                WHERE {
                                             ?tvseries wdt:P161 ?castmember.
                                              {
                                                  SELECT DISTINCT  ?castmember 
                                                      WHERE { 
                                                              ?tvseries wdt:P31 wd:Q5398426;
                                                                        wdt:P161 wd:Q3454165.

                                                              ?tvseries wdt:P161 ?castmember.
                                                            }
                                             }
                                     }
                 }
                 MINUS{
                        SELECT DISTINCT ?castmember
                                WHERE {
                                             ?tvseries wdt:P161 ?castmember.
                                              {
                                                  SELECT DISTINCT  ?castmember 
                                                      WHERE { 
                                                          ?tvseries wdt:P31 wd:Q5398426;
                                                                    wdt:P161 wd:Q3454165.

                                                          ?tvseries wdt:P161 ?castmember.

                                                            }
                                             }
                                      }
                              }
                 
                ?tvseries <http://schema.org/name> ?tvseriesName .
                ?castmember <http://schema.org/name> ?castmemberName .
             }
               

order by ?tvseriesName         
limit 500

"""

print("Results")
run_query(queryString)

Results
[('castmemberName', 'Sarah Shahi'), ('tvseriesName', '.exe')]
[('castmemberName', 'Brett Cullen'), ('tvseriesName', '.exe')]
[('castmemberName', 'Ebon Moss-Bachrach'), ('tvseriesName', '.exe')]
[('castmemberName', 'Kevin Chapman'), ('tvseriesName', '.exe')]
[('castmemberName', 'Jim Caviezel'), ('tvseriesName', '.exe')]
[('castmemberName', 'Michael Emerson'), ('tvseriesName', '.exe')]
[('castmemberName', 'Emmanuelle Chriqui'), ('tvseriesName', '100 Girls')]
[('castmemberName', 'Aimee Graham'), ('tvseriesName', '100 Girls')]
[('castmemberName', 'Jonathan Tucker'), ('tvseriesName', '100 Girls')]
[('castmemberName', 'Jaime Pressly'), ('tvseriesName', '100 Girls')]
[('castmemberName', 'Marissa Ribisi'), ('tvseriesName', '100 Girls')]
[('castmemberName', 'Katherine Heigl'), ('tvseriesName', '100 Girls')]
[('castmemberName', 'Larisa Oleynik'), ('tvseriesName', '100 Girls')]
[('castmemberName', 'James DeBello'), ('tvseriesName', '100 Girls')]
[('castmemberName', 'Kristina Anapau'), ('t

500